In [ ]:
API_KEY = ""

In [ ]:
!apt install rustc

In [ ]:
!wget https://github.com/THUDM/CodeGeeX/raw/f66205b5f615a4eead9c26d7ec297e14738ea18d/codegeex/benchmark/humaneval-x/rust/data/humaneval_rust.jsonl.gz
!gunzip humaneval_rust.jsonl.gz

In [ ]:
import requests

def request_completion(prompt):
    endpoint = 'https://api.together.xyz/inference'
    res = requests.post(endpoint, json={
        "model": "togethercomputer/llama-2-13b-chat",
        "max_tokens": 512,
        "prompt": prompt,
        "request_type": "language-model-inference",
        "temperature": 0.4,
        "top_p": 0.7,
        "top_k": 50,
        "repetition_penalty": 1
    }, headers={
        "Authorization": f"Bearer {API_KEY}",
    })
    return res.json()['output']['choices'][0]['text']

In [ ]:
import subprocess

def compile_and_execute_rust(rust_code):
    # Create a Rust source code file from the provided code string
    with open("temp.rs", "w") as rust_file:
        rust_file.write(rust_code)

    binary_path = "rust_program"  # Name of the compiled binary

    # Compile the Rust code
    compile_command = ["rustc", "temp.rs", "-o", binary_path]

    try:
        subprocess.run(compile_command, check=True)
    except subprocess.CalledProcessError as e:
        print("Compilation failed:", e)
        return False

    # Execute the compiled binary
    execute_command = ["./" + binary_path]

    try:
        subprocess.run(execute_command, check=True)
        print("Rust program executed successfully.")
        return True
    except subprocess.CalledProcessError as e:
        print("Rust program execution failed:", e)
        return False

In [ ]:
import json

jsonl_file_path = "humaneval_rust.jsonl"

with open(jsonl_file_path, 'r') as file:
    for line in file:
        data = json.loads(line)
        #print(dict.keys(data))
        print(data['task_id'] + ":")
        print("Generating...")
        code = request_completion(data['declaration']+data['prompt'])
        main = "\nfn main(){ \n } \n"
        test_string = main + data['declaration'] + data['prompt'] + code + data['test']
        #print(test_string)
        with open(data['task_id'].replace("/", "-") + ".rs", "w") as file:
          file.write(test_string)
        print("Evaluating...")
        compile_and_execute_rust(test_string)
